# Entrenamiento de YOLOv8 en Google Colab

Este notebook está adaptado para entrenar un modelo YOLOv8, guardando los resultados en Google Drive y con capacidad de reanudar el entrenamiento si la sesión se interrumpe.

### 1. Montar Google Drive

Conecta el notebook a tu Google Drive. Aquí es donde leeremos el dataset y guardaremos los resultados y checkpoints.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 2. Instalar Librerías

Instalamos `ultralytics` para YOLOv8.

In [ ]:
!pip install ultralytics

### 3. Configuración del Entrenamiento

**ACCIÓN REQUERIDA:** Ajusta las rutas y parámetros en la siguiente celda según tu configuración.

In [ ]:
import os
from ultralytics import YOLO

# --- PARÁMETROS DE CONFIGURACIÓN ---
# Asegurarse de que CUDA_VISIBLE_DEVICES esté configurado para usar la primera GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Ruta base en tu Google Drive donde tienes el proyecto
# DEBES AJUSTAR ESTA RUTA a donde hayas guardado tu dataset
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/mod03-projects/project-xii-computer-vision' # Asegúrate de que esta ruta sea correcta

# --- Configuración del Dataset Comprimido ---# Nombre del archivo .tar.gz del dataset reducido (debe estar en la misma carpeta que DRIVE_PROJECT_PATH)
COMPRESSED_DATASET_NAME = 'dataset_v1_yolov8_reduced.tar.gz'
COMPRESSED_DATASET_PATH = os.path.join(DRIVE_PROJECT_PATH, COMPRESSED_DATASET_NAME)

# Directorio donde se descomprimirá el dataset dentro de Drive (persistente)
DECOMPRESSION_TARGET_DIR = DRIVE_PROJECT_PATH # Descomprimir directamente en la raíz del proyecto en Drive
# Ruta a la carpeta del dataset una vez descomprimido
DATASET_EXTRACTED_PATH = os.path.join(DECOMPRESSION_TARGET_DIR, 'dataset_v1_yolov8_reduced')

# Ruta al archivo YAML del dataset (APUNTANDO A LA VERSIÓN DESCOMPRIMIDA)
DATASET_YAML_PATH = os.path.join(DATASET_EXTRACTED_PATH, 'data.yaml')

# Modelo, epochs y nombre del run
MODEL_NAME = 'yolov8m.pt'
EPOCHS = 80
RUN_NAME = 'yolov8m_colab_80_epochs'

# Parámetros de rendimiento para Colab
BATCH_SIZE = 8
IMG_SIZE = 640
WORKERS = 4

# --- FIN DE LA CONFIGURACIÓN ---
# Asegurarse de que la carpeta de resultados exista en Drive
os.makedirs(os.path.join(DRIVE_PROJECT_PATH, 'training_results'), exist_ok=True)

# --- Descompresión del Dataset ---print(f"Descomprimiendo {COMPRESSED_DATASET_NAME} a {DECOMPRESSION_TARGET_DIR}...")
os.makedirs(DECOMPRESSION_TARGET_DIR, exist_ok=True) # Asegurarse de que la carpeta de destino exista
!tar -xzf {COMPRESSED_DATASET_PATH} -C {DECOMPRESSION_TARGET_DIR}
print("Descompresión completada.")

### 4. Entrenar el Modelo

Esta celda inicia o reanuda el entrenamiento. Gracias a `resume=True`, si el script se detiene, puedes volver a ejecutar esta celda y continuará donde lo dejó.

In [ ]:
# Cargar el modelo base (YOLO descargará 'yolov8m.pt' si no existe)
model = YOLO(MODEL_NAME)

# Iniciar el entrenamiento
results = model.train(
    # --- Rutas y Datos ---
    data=DATASET_YAML_PATH,
    project=os.path.join(DRIVE_PROJECT_PATH, 'training_results'), # Guarda los resultados en Drive
    name=RUN_NAME,
    
    # --- Parámetros de Entrenamiento ---    epochs=EPOCHS,
    resume=True,  # <-- ¡LA CLAVE! Reanuda el entrenamiento si encuentra un checkpoint
    
    # --- Parámetros de Rendimiento ---    batch=BATCH_SIZE,
    imgsz=IMG_SIZE,
    workers=WORKERS,
    
    # --- Opcionales ---    save_json=True,
    save_hybrid=True,
    plots=True,
    device='0' # Forzar el uso de la GPU 0
)

print('--- ENTRENAMIENTO FINALIZADO ---')
print(f'Resultados guardados en: {results.save_dir}')

### 5. Evaluar el Modelo Final

Una vez finalizado el entrenamiento completo, puedes ejecutar esta celda para ver las métricas de validación del mejor modelo.

In [ ]:
# Cargar el mejor modelo guardado durante el entrenamiento
best_model_path = os.path.join(DRIVE_PROJECT_PATH, 'training_results', RUN_NAME, 'weights', 'best.pt')
model = YOLO(MODEL_NAME)

# Evaluar en el conjunto de validación
metrics = model.val(project=os.path.join(DRIVE_PROJECT_PATH, 'training_results'), name=f'{RUN_NAME}_val')

print('--- MÉTRICAS DE VALIDACIÓN ---')
print(f'mAP50-95: {metrics.box.map:.4f}')
print(f'   mAP50: {metrics.box.map50:.4f}')
print(f'   mAP75: {metrics.box.map75:.4f}')